In [1]:
from pathlib import Path
import pandas as pd
ROOT = Path('../..')
ROOT.resolve()

PosixPath('/Users/lukestrange/Code/housing')

In [2]:
data = pd.read_csv(ROOT / 'raw/affordable-homes/affordable_homes_open_data_202223')

In [3]:
# Remove un-needed columns
data.drop(columns=['LA code', 'LA name', 'District name', 'Metropolitan code', 'Metropolitan name', 'Region code', 'Region name', 'LA type 202223'], inplace=True)

In [4]:
# Rename column
data.rename(columns={'LA code 202223': 'geography_code', 'LA name 202223': 'geography_name'}, inplace=True)

In [5]:
# We are only interested in completions for now
data = data.loc[data.Completions=='Completion']

In [6]:
# Sum values per year, per local authority, per tenure type
local_authority = data.groupby(['geography_code', 'geography_name', 'Year', 'Tenure']).sum(numeric_only=True).reset_index()

local_authority_wide = local_authority.pivot(index=['geography_code', 'geography_name', 'Year'], columns='Tenure', values='Units')

# Add a column for all affordable homes, which is the sum of all the tenure types.
local_authority_wide['All afforable'] = local_authority_wide.sum(axis=1)

In [7]:
# Calculate the values for the whole of England
all_england = local_authority.groupby(['Year', 'Tenure']).sum(numeric_only=True).reset_index()

# Pivot to wide format
all_england_wide = all_england.pivot(index='Year', columns='Tenure', values='Units')

# Add a column for all affordable homes, which is the sum of all the tenure types.
all_england_wide['All afforable'] = all_england_wide.sum(axis=1)

# Add the geo code for England and append it to the index.
all_england_wide['geography_code'] = 'E92000001'
all_england_wide['geography_name'] = 'England'
all_england_wide.set_index(['geography_code', 'geography_name'], append=True, inplace=True)
all_england_wide = all_england_wide.reorder_levels(['geography_code', 'geography_name', 'Year'])

In [8]:
# Concatenate the dataframes
combined = pd.concat([local_authority_wide, all_england_wide])

In [9]:
# Write the files to CSV
combined.to_csv(ROOT / 'data/affordable-homes/by_tenure.csv')
combined.to_parquet(ROOT / 'data/affordable-homes/by_tenure.parquet')
# all_england.to_csv('data/affordable-homes/by_tenure_england.csv')

In [10]:
combined

Tenure                                 Affordable Home Ownership  \
geography_code geography_name Year                                 
E06000001      Hartlepool     1991-92                       13.0   
                              1992-93                       29.0   
                              1993-94                       20.0   
                              1994-95                       22.0   
                              1995-96                       23.0   
...                                                          ...   
E92000001      England        2018-19                     2460.0   
                              2019-20                     2108.0   
                              2020-21                     1134.0   
                              2021-22                     1047.0   
                              2022-23                     1172.0   

Tenure                                 Affordable Rent  First Homes  \
geography_code geography_name Year                                    
E06000001      Hartlepool     1991-92              NaN          NaN   
                              1992-93              NaN          NaN   
                              1993-94              NaN          NaN   
                              1994-95              NaN          NaN   
                              1995-96              NaN          NaN   
...                                                ...          ...   
E92000001      England        2018-19          28957.0          NaN   
                              2019-20          28259.0          NaN   
                              2020-21          23786.0          NaN   
                              2021-22          26398.0         35.0   
                              2022-23          24547.0       1021.0   

Tenure                                 Intermediate Rent  \
geography_code geography_name Year                         
E06000001      Hartlepool     1991-92                NaN   
                              1992-93                NaN   
                              1993-94                NaN   
                              1994-95                NaN   
                              1995-96                NaN   
...                                                  ...   
E92000001      England        2018-19             1383.0   
                              2019-20             1748.0   
                              2020-21             2026.0   
                              2021-22             1477.0   
                              2022-23             2701.0   

Tenure                                 London Affordable Rent  \
geography_code geography_name Year                              
E06000001      Hartlepool     1991-92                     NaN   
                              1992-93                     NaN   
                              1993-94                     NaN   
                              1994-95                     NaN   
                              1995-96                     NaN   
...                                                       ...   
E92000001      England        2018-19                  1002.0   
                              2019-20                  1797.0   
                              2020-21                  2102.0   
                              2021-22                  3101.0   
                              2022-23                  4296.0   

Tenure                                 Shared Ownership  Social Rent  Unknown  \
geography_code geography_name Year                                              
E06000001      Hartlepool     1991-92               NaN        134.0      NaN   
                              1992-93               NaN        204.0      NaN   
                              1993-94               NaN        123.0      NaN   
                              1994-95               NaN        149.0      NaN   
                              1995-96               NaN        134.0      NaN 

# TODO 
Use a lookup to go from LAs to met counties and regions.